# TODO
- Dataset class?
- Dataloader
- network
- train
- val
- visualizer

In [1]:
from glob import glob
import json
import math
import os

from PIL import Image
import requests
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'retina'

import ipywidgets as widgets
from IPython.display import display, clear_output

import torch
from torch import nn
from torchvision import io
import torchvision.transforms as T
torch.set_grad_enabled(False);

## Data

In [2]:
# Pathing
# =======
BDD_DIR = os.environ['HOME'] + '/.Data/BDD/bdd100k'
IMAGE_DIR = BDD_DIR + '/images/100k'
TRAIN_IMAGE_DIR = IMAGE_DIR + '/train'
VAL_IMAGE_DIR   = IMAGE_DIR + '/val'
DET_LABELS_DIR = BDD_DIR + '/labels/detection20'


# Files
# =====
DET_LABELS_TRAIN = DET_LABELS_DIR + '/det_v2_train_release.json'
DET_LABELS_VAL   = DET_LABELS_DIR + '/det_v2_val_release.json'

In [3]:
def read_json(path):
    with open(path) as jfile:
        return json.load(jfile)

labels_train = read_json(DET_LABELS_TRAIN)
labels_val   = read_json(DET_LABELS_VAL)

In [4]:
print(f"{len(labels_train)} {len(labels_val)}")

69863 10000


In [5]:
labels_train[0]

{'videoName': '0000f77c-6257be58',
 'name': '0000f77c-6257be58.jpg',
 'index': 0,
 'timestamp': 10000,
 'labels': [{'category': 'traffic light',
   'poly2d': None,
   'box3d': None,
   'box2d': {'x2': 1156.978645,
    'y1': 133.184488,
    'x1': 1125.902264,
    'y2': 210.875445},
   'manualShape': True,
   'attributes': {'occluded': False,
    'truncated': False,
    'trafficLightColor': [1, 'G']},
   'id': 0},
  {'category': 'traffic light',
   'poly2d': None,
   'box3d': None,
   'box2d': {'x2': 1191.50796,
    'y1': 136.637417,
    'x1': 1156.978645,
    'y2': 210.875443},
   'manualShape': True,
   'attributes': {'occluded': False,
    'truncated': False,
    'trafficLightColor': [1, 'G']},
   'id': 1},
  {'category': 'traffic sign',
   'poly2d': None,
   'box3d': None,
   'box2d': {'x2': 1170.79037,
    'y1': 211.122087,
    'x1': 1105.66915985699,
    'y2': 233.566141},
   'manualShape': True,
   'attributes': {'occluded': False,
    'truncated': False,
    'trafficLightColor': 

In [6]:
len(labels_train)

69863

In [7]:
#TRAIN_IMAGE_DIR = IMAGE_DIR + '/train'
#VAL_IMAGE_DIR   = IMAGE_DIR + '/val'

images_train = sorted(glob(TRAIN_IMAGE_DIR + '/*.jpg'))
images_val   = sorted(glob(VAL_IMAGE_DIR + '/*.jpg'))

In [8]:
len(images_train)

70000

In [9]:
foo = io.read_image(images_train[0])

In [10]:
foo

tensor([[[101,  46,  33,  ..., 152, 152, 152],
         [ 96,  45,  37,  ..., 152, 152, 152],
         [ 97,  49,  38,  ..., 152, 152, 152],
         ...,
         [ 26,  26,  26,  ...,  13,  13,  13],
         [ 26,  26,  26,  ...,  13,  13,  13],
         [ 26,  26,  26,  ...,  14,  14,  14]],

        [[105,  50,  37,  ..., 199, 199, 199],
         [100,  49,  41,  ..., 199, 199, 199],
         [101,  53,  42,  ..., 199, 199, 199],
         ...,
         [ 29,  29,  29,  ...,  13,  13,  13],
         [ 29,  29,  29,  ...,  13,  13,  13],
         [ 29,  29,  29,  ...,  14,  14,  14]],

        [[108,  53,  40,  ..., 229, 229, 229],
         [103,  52,  44,  ..., 229, 229, 229],
         [104,  56,  45,  ..., 229, 229, 229],
         ...,
         [ 36,  36,  36,  ...,  15,  15,  15],
         [ 36,  36,  36,  ...,  15,  15,  15],
         [ 36,  36,  36,  ...,  16,  16,  16]]], dtype=torch.uint8)

In [11]:
foo.shape

torch.Size([3, 720, 1280])

## DATALOADER
https://pytorch.org/docs/stable/data.html